In [5]:
# pip install transformers datasets torch

In [6]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset

# 加载预训练模型和分词器
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# 加载数据集（例如 IMDb 数据集）
dataset = load_dataset("imdb")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [8]:
def preprocess_data(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# 预处理数据
tokenized_datasets = dataset.map(preprocess_data, batched=True)



Map: 100%|██████████| 25000/25000 [00:02<00:00, 9342.90 examples/s]


In [9]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 50000
    })
})

In [10]:
train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(2000))
test_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

train_dataset = train_dataset.remove_columns(["text"])
test_dataset = test_dataset.remove_columns(["text"])
train_dataset


Dataset({
    features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 2000
})

In [11]:
test_dataset

Dataset({
    features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1000
})

In [12]:
from transformers import TrainingArguments, Trainer

# 定义训练参数
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    learning_rate=2e-5
)

# 定义 Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

# 开始训练
trainer.train()


/home/hnyx/anaconda3/envs/hug_py310/lib/python3.10/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
  0%|          | 0/375 [00:00<?, ?it/s]/home/hnyx/anaconda3/envs/hug_py310/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
                                                 
 33%|███▎      | 125/375 [00:39<01:03,  3.95it/s]

{'eval_loss': 0.3649536669254303, 'eval_runtime': 6.7448, 'eval_samples_per_second': 148.262, 'eval_steps_per_second': 9.34, 'epoch': 1.0}


/home/hnyx/anaconda3/envs/hug_py310/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
                                                 
 67%|██████▋   | 250/375 [01:18<00:31,  3.95it/s]

{'eval_loss': 0.34758254885673523, 'eval_runtime': 6.7348, 'eval_samples_per_second': 148.483, 'eval_steps_per_second': 9.354, 'epoch': 2.0}


/home/hnyx/anaconda3/envs/hug_py310/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
100%|██████████| 375/375 [01:51<00:00,  3.96it/s]/home/hnyx/anaconda3/envs/hug_py310/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
                                                 
100%|██████████| 375/375 [01:59<00:00,  3.96it/s]

{'eval_loss': 0.34204450249671936, 'eval_runtime': 6.7306, 'eval_samples_per_second': 148.574, 'eval_steps_per_second': 9.36, 'epoch': 3.0}


100%|██████████| 375/375 [02:00<00:00,  3.12it/s]

{'train_runtime': 120.11, 'train_samples_per_second': 49.954, 'train_steps_per_second': 3.122, 'train_loss': 0.2514924112955729, 'epoch': 3.0}


TrainOutput(global_step=375, training_loss=0.2514924112955729, metrics={'train_runtime': 120.11, 'train_samples_per_second': 49.954, 'train_steps_per_second': 3.122, 'total_flos': 1578666332160000.0, 'train_loss': 0.2514924112955729, 'epoch': 3.0})

In [13]:
results = trainer.evaluate()
print("Evaluation results:", results)
model.save_pretrained("./text_classification_model")
tokenizer.save_pretrained("./text_classification_model")


/home/hnyx/anaconda3/envs/hug_py310/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
100%|██████████| 63/63 [00:06<00:00,  9.40it/s]


Evaluation results: {'eval_loss': 0.34204450249671936, 'eval_runtime': 6.7529, 'eval_samples_per_second': 148.085, 'eval_steps_per_second': 9.329, 'epoch': 3.0}


('./text_classification_model/tokenizer_config.json',
 './text_classification_model/special_tokens_map.json',
 './text_classification_model/vocab.txt',
 './text_classification_model/added_tokens.json',
 './text_classification_model/tokenizer.json')

In [ ]:
# pip install fastapi uvicorn


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 142.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [5]:
from fastapi import FastAPI
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModelForSeq2SeqLM
import torch

app = FastAPI()

# 加载模型和分词器
# model_name = "./text_classification_model"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForSequenceClassification.from_pretrained(model_name)
model_name = "Langboat/mengzi-t5-base"
tokenizer = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

@app.post("/predict")
async def predict(text: str):
    inputs = tokenizer(text, return_tensors="pt", padding="max_length", truncation=True, max_length=128)
    outputs = model(**inputs)
    prediction = torch.argmax(outputs.logits, dim=-1).item()
    return {"text": text, "label": prediction}


In [ ]:
# 启动服务
uvicorn model_deploy.text.app:app --reload --host 0.0.0.0 --port 8000
# 测试部署
curl -X POST "http://127.0.0.1:8000/predict" -H "Content-Type: application/json" -d '{"text": "This movie was amazing!"}'


In [3]:
from ultralyticsplus import YOLO, render_result

# load model
model = YOLO('ultralyticsplus/yolov8s')

# set model parameters
model.overrides['conf'] = 0.25  # NMS confidence threshold
model.overrides['iou'] = 0.45  # NMS IoU threshold
model.overrides['agnostic_nms'] = False  # NMS class-agnostic
model.overrides['max_det'] = 1000  # maximum number of detections per image

# set image
image = '/home/hnyx/workspace/model_deploy/data/bus.jpg'

# perform inference
results = model.predict(image)

# observe results
print(results[0].boxes)
render = render_result(model=model, image=image, result=results[0])
render.show()
type(model)


Ultralytics YOLOv8.0.8 🚀 Python-3.10.16 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce RTX 4090 D, 24203MiB)
Fusing layers... 
YOLOv8s summary: 168 layers, 11156544 parameters, 0 gradients, 28.6 GFLOPs


Ultralytics YOLO <class 'ultralytics.yolo.engine.results.Boxes'> masks
type: <class 'torch.Tensor'>
shape: torch.Size([5, 6])
dtype: torch.float32


ultralyticsplus.ultralytics_utils.YOLO

In [8]:
from transformers import pipeline
 
# 创建一个问答模型的实例
qa_model = pipeline('question-answering')
 
# 定义问题和文本，其中文本包含了问题的答案
question = "你是谁？"
text = "我是一个人工智能助手。"
 
# 调用问答模型
answer = qa_model(question=question, context=text)
 
# 打印结果
print(answer)

/home/hnyx/anaconda3/envs/hug_py310/lib/python3.10/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/home/hnyx/anaconda3/envs/hug_py310/lib/python3.10/site-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submi

{'score': 0.06692501902580261, 'start': 4, 'end': 11, 'answer': '人工智能助手。'}


In [12]:
import time
from transformers import pipeline, Pipeline
name = "distilbert/distilbert-base-uncased-finetuned-sst-2-english"
# 加载预训练的文本分类模型
classifier = pipeline("text-classification", model=name)
since = time.time()
# 进行推理
result = classifier("I hate you.")
print(result)
print(time.time()-since)


Device set to use cuda:0


[{'label': 'NEGATIVE', 'score': 0.9992952346801758}]
0.0026552677154541016
